In [4]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [5]:

pip install spotipy

Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip install urllib3 --upgrade 
%pip install requests --upgrade 
%pip install spotipy --upgrade

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [128]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import os
import numpy as np
import timeit
import pandas as pd
from operator import itemgetter
import datetime as dt
from collections import Counter
from operator import itemgetter
from itertools import chain

os.environ['SPOTIPY_CLIENT_ID'] = '3191e7e8b04e46c1af64f49bcdd020be'
os.environ['SPOTIPY_CLIENT_SECRET'] = '88dc925eff0d4006bb6d0b9b2401ac59'

In [8]:
auth_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(auth_manager=auth_manager)

In [9]:
import json
with open('mpd_slice_0_999.json') as json_file:
    data = json.load(json_file)

In [134]:
# calculate artist concentration value for a playlist
# low value means high concentration - maybe rename to artist diversity?
# input: 2D array of artists for each song
# output: real value between 0 and 1
def get_artist_div(artists):
    #print('get_artist_div')
    #print(artists)
    unique_artists = len(set(artists))
    total_appearances = len(artists)
    artist_div = unique_artists / total_appearances
    return artist_div

In [135]:
MS_IN_DAY = 86400
def get_artists_and_dates(uris):
    # do the batching above this- here we assume that there are no more than 50 URIs
    # i could put an assert statement here but it would slow the code down
    # assert(len(uris) <= 50)
    artists = {}
    dates = {}
    tracks = sp.tracks(uris)['tracks']
    none_uris = []
    for uri, track in zip(uris, tracks):
        if track is None:
            print(track)
            none_uris.append(uri)
        else:
            artists[uri] = pd.DataFrame(track['artists'])['name'].values.tolist()
            #print(track['artists'])
            #print(artists[uri])
            date = track['album']['release_date']
            while len(date) < 10:
                date += '-01'
            ms_date = dt.datetime.timestamp(dt.datetime.strptime(date,"%Y-%m-%d"))
            dates[uri] = ms_date / MS_IN_DAY
            
    return artists, dates, none_uris

In [120]:
# get all audio features for a given playlist
def get_playlist_audio_features_batch(playlist_uris):
    remainder = len(playlist_uris) % 100
    all_features = []
    if remainder > 0: 
        all_features = sp.audio_features(playlist_uris[:remainder])
    while remainder < len(playlist_uris):
        all_features += sp.audio_features(playlist_uris[remainder:remainder+100])
        remainder += 100
    
    none_uris = []
    remainder = len(playlist_uris) % 50
    artists, dates = {}, {}
    if remainder > 0:
        artists, dates, none_uris_iter = get_artists_and_dates(playlist_uris[:remainder])
        none_uris += none_uris_iter
    while remainder < len(playlist_uris):
        next_artists, next_dates, none_uris_iter = get_artists_and_dates(playlist_uris[remainder:remainder+50])

        artists.update(next_artists)
        dates.update(next_dates)
        remainder += 50
        none_uris += none_uris_iter

    # for i in range(len(all_features)):
    #     print(i)
    #     print(all_features[i]['uri'])
    #     print(artists.keys()[i])
    return all_features, artists, dates, none_uris

In [112]:
# splits out uris that we have already called the api for
# also eliminates duplicate songs within a playlist
def deduplicate_uris(playlist_uris, used_tracks_by_uri):
    duplicates = np.intersect1d(playlist_uris, np.array(list(used_tracks_by_uri.keys())))
    new_uris = np.setdiff1d(playlist_uris, np.array(list(used_tracks_by_uri.keys())))
    return duplicates, new_uris#, count_ids

In [171]:
# make sure artists and dates get added to used_tracks_by_uri entries
# 

# calculate aggregate audio features for a playlist
# currently only focused on audio features, I'll deal with artist and release date later
def get_playlist_aggregate_features(tracks, used_tracks_by_uri, used_artists_by_uri, used_dates_by_uri):

    tracks = pd.DataFrame(tracks)
    duplicates, new_uris = deduplicate_uris(tracks['track_uri'], used_tracks_by_uri) # deduplicate here

    all_features, artists, dates, none_uris = get_playlist_audio_features_batch(new_uris)

    # add duplicates to this playlist for aggregate metric calculation
    dup_artists = []
    dup_dates = []
    if len(duplicates) > 1:
        all_features += list(itemgetter(*duplicates)(used_tracks_by_uri))
        dup_artists = list(itemgetter(*duplicates)(used_artists_by_uri)) # artists is a dict - update it accordingly
        dup_dates = list(itemgetter(*duplicates)(used_dates_by_uri))
    elif len(duplicates) == 1:
        all_features.append(used_tracks_by_uri[duplicates[0]])
        dup_artists = used_artists_by_uri[duplicates[0]]
        print(used_dates_by_uri[duplicates[0]])
        dup_dates = list(used_dates_by_uri[duplicates[0]])

    #print(all_features[19])
    all_features = pd.DataFrame(all_features)

    # option 1 - add artists and dates into all_features
    # option 2 - keep artists and dates separate and do a used_artists_by_uri dict
    # all_features['artists'] = artists.values()
    # all_features['date'] = dates.values()
    # for row in all_features:
    #     print(row['uri'])
    #     print(row['artists'])
    #     print(artists[row['uri']])
    #     print()


    # all_features.set_index(all_features['uri'])
    # print(all_features)
    # all_features = all_features.to_dict('index')
    # print(all_features)
    
    # update list of already used tracks
    for i, uri in zip(range(len(new_uris)), new_uris):
        if uri not in none_uris:
            used_tracks_by_uri[uri] = all_features.loc[all_features['uri'] == uri].to_dict('records')[0]
            #print(used_tracks_by_uri[uri])
            #artist_dict[uri] = artists[uri].values
            #dates_dict[uri] = dates[uri]
            #print(used_tracks_by_uri[uri])
            #print(artists[uri])
            #print(artists[uri].to_numpy())
            used_artists_by_uri[uri] = artists[uri] #.insert(1, 'artists', artists[uri].to_numpy(), True)
            used_dates_by_uri[uri] = dates[uri] #.insert(1, 'date', dates[uri], True)
            #print(used_tracks_by_uri[uri]['artists'])

            # print(all_features.loc[all_features['uri'] == uri])
            
            
    
    
    #print(artists)
    #all_features = pd.DataFrame(all_features)
    #print(all_features['artists'])
    #all_features[weird_uri]['artists'] = 'uh'
    #artists = np.concatenate(all_features['artists']).ravel()
    #print('yay?')


    # #print(all_features[19])
    # all_features = pd.DataFrame(all_features)
    #print(all_features)
    #print(all_features['artists'])
    #artists_list = np.concatenate(all_features['artists']).ravel()# + artists.values()


    # calculate aggregate features
    feature_list = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']
    aggregate_features = {}
    for feature in feature_list:
        all_by_feature = all_features[feature]
        aggregate_features[feature] = np.mean(all_by_feature)
        aggregate_features[feature+'_var'] = np.var(all_by_feature)

    #print(list(chain(*artists.values())))
    aggregate_features['date'] = np.mean(list(dates.values()) + dup_dates)
    aggregate_features['date_var'] = np.var(list(dates.values()) + dup_dates)
    aggregate_features['artist_div'] = get_artist_div(list(chain(*artists.values())) + list(chain(*dup_artists)))
    return aggregate_features

In [172]:
# build the aggregate dataset!
def build_dataset(playlists):
    aggregate_dataset = []
    used_tracks_by_uri = {}
    used_artists_by_uri = {}
    used_dates_by_uri = {}
    for playlist in playlists:
        aggregate_dataset.append(get_playlist_aggregate_features(playlist['tracks'], used_tracks_by_uri, used_artists_by_uri, used_dates_by_uri))
    return aggregate_dataset

In [174]:
#[497:498] has a uri that returns None from the API in it, uri 19 in the list
# somewhere after it has a mean of empty slice situation happening
dataset = pd.DataFrame(build_dataset(data['playlists'][0:100]))


TypeError: 'float' object is not iterable

In [ ]:
dataset.to_csv(r'csvdata_nodup_10.csv')
print(dataset)

In [170]:
times = [1, 10, 100, 1000]
for i in times:
    t = timeit.Timer('build_dataset(data[\'playlists\'][:i])', 'from __main__ import build_dataset, data, i')
    print('Time: ' + str(t.timeit(1)))

Time: 0.4190453339997475
Time: 3.8613711670004705


TypeError: can only concatenate list (not "float") to list

In [ ]:
#207 dates mean of empty slice